<a href="https://colab.research.google.com/github/evilangelsaahi/openAIgym/blob/main/openAIgym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install gym
#!pip install --user tf-agents
#!pip install tensorflow
#!pip install ray
#!pip install stable_baselines3
#!pip install pyglet
#!pip install keras

### Import gym library

In [1]:
import gym
import random

In [ ]:
import numpy as np

In [ ]:
from keras.models import Sequential

In [ ]:
from keras.layers import Dense

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
env_name = 'CartPole-v0'
env = gym.make(env_name)
env.reset()
goal_steps = 200
score_requirement = 195
intial_games = 100

### set the enviornment to its initial state and return 4 values in array
![Screen Shot 2022-06-10 at 3.42.13 PM.png](attachment:654c6303-9013-4f0c-ae2c-6c5751de37cd.png)

In [ ]:
obs = env.reset()
obs

### observation_space returns the information about the environment space
    # Box data type i.e. for continous observation
    # returned Box object represents
    # Box([[lower range of obs], [upper range obs], (number of dimensions), data type])
    

In [ ]:
env.observation_space

### action_space returns the Discrete object 
### sequence of integers, reprents the actions
![Screen Shot 2022-06-10 at 3.45.03 PM.png](attachment:1bfb2d6b-c3cf-4723-8b37-9fd8dde1e890.png)

In [ ]:
env.action_space

In [ ]:
#env.render()

### To take an action we need step() function
    # step function returns 4 parameters
    # [[env state], reward, terminal state(bool),]

In [ ]:
#env.step(0)

In [ ]:
#steps, reward, done, _ = env.step(0)


### Below 2 blocks are a simple demonstartion of pushing cart to one side

In [ ]:
for episode in range(1, 11):
    score = 0
    state = env.reset()
    done = False
    
    while not done:
      env.render()
      action = env.action_space.sample()
      n_state, reward, done, info = env.step(action)
      score += reward
      
    print('Episode:', episode, 'Score:', score)



In [ ]:
# to close the window uncomment the command below

env.close()

In [ ]:
import gym
env = gym.make('CartPole-v0')
env.reset()
for step_index in range(1000):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print("Step {}:".format(step_index))
    print("action: {}".format(action))
    print("observation: {}".format(observation))
    print("reward: {}".format(reward))
    print("done: {}".format(done))
    print("info: {}".format(info))
    if done:
        break

### Training data

In [ ]:
def model_data_preparation():
    training_data = []
    accepted_scores = []
    for game_index in range(intial_games):
        score = 0
        game_memory = []
        previous_observation = []
        for step_index in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(previous_observation) > 0:
                game_memory.append([previous_observation, action])
                
            previous_observation = observation
            score += reward
            if done:
                break
            
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                training_data.append([data[0], output])
        
        env.reset()

    print(accepted_scores)
    
    return training_data

### Building and model training

In [ ]:
def build_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(128, input_dim=input_size, activation='relu'))
    model.add(Dense(52, activation='relu'))
    model.add(Dense(output_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam())
    return model

In [ ]:
def train_model(training_data):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]))
    y = np.array([i[1] for i in training_data]).reshape(-1, len(training_data[0][1]))
    model = build_model(input_size=len(X[0]), output_size=len(y[0]))
    
    model.fit(X, y, epochs=10)
    return model


In [ ]:
training_data = model_data_preparation()
trained_model = train_model(training_data)

In [ ]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    prev_obs = []
    for step_index in range(goal_steps):
      #env.render()
      if len(prev_obs)==0:
            action = random.randrange(0,2)
      else:
            action = np.argmax(trained_model.predict(prev_obs.reshape(-1, len(prev_obs)))[0])
        
      choices.append(action)
      new_observation, reward, done, info = env.step(action)
      prev_obs = new_observation
      score+=reward
      if done:
          break

    env.reset()
    scores.append(score)

print(scores)
print('Average Score:', sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
